In [2]:
''' 任务：
演示Tensorflow Hub和keras进行迁移学习的基本应用

使用包含 Internet Movie Database 中的 50,000 条电影评论文本的 IMDB 数据集。先将这些评论分为两组，其中 25,000 条用于训练，另外 25,000 条用于测试。训练组和测试组是均衡的，也就是说其中包含相等数量的正面评价和负面评价。

下载IMDB 数据集
探索数据
构建模型
    损失函数与优化器
训练模型
评估模型
进一步阅读

使用 tf.keras（一个在 TensorFlow 中用于构建和训练模型的高级 API）和 tensorflow_hub（一个用于在一行代码中从 TFHub 加载训练模型的库）。
'''

import os
import numpy as np

import tensorflow as tf
import tensorflow_hub as hub
import tensorflow_datasets as tfds

print("Version:", tf.__version__)
print("Eager mode:", tf.executing_eagerly())
print("Hub version:", hub.__version__)
print("GPU is", "available" if tf.config.list_physical_devices("GPU") else "NOT AVAILABLE")


E:\LocalRepository\Anaconda3_envs\deeplearning\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Version: 2.13.0
Eager mode: True
Hub version: 0.15.0
GPU is NOT AVAILABLE


In [3]:
# SECTION: 下载IMDB数据集
'''
IMDB数据 可以在Tensorflow数据集处获取，
'''
train_data, validation_data, test_data = tfds.load(
    name='imdb_reviews',
    split=('train[:60%]', 'train[60%:]', 'test'),
    as_supervised=True
)

# SECTION: 探索数据

'''
了解数据格式，每个句子都是一个代表电影评论的句子和一个相应的标签
句子未经任何预处理，标签是一个整数值（0或1）， 其中0表示负面评价，而1表示正面评价

打印前十个样本
'''

train_examples_batch, train_labels_batch = next(iter(train_data.batch(10)))
print(train_examples_batch)
print(train_labels_batch)

Dl Completed...: 0 url [00:00, ? url/s]
Generating splits...:   0%|          | 0/3 [00:00<?, ? splits/s]
Generating train examples...: 0 examples [00:00, ? examples/s]
Generating train examples...: 1 examples [00:01,  1.29s/ examples]
Generating train examples...: 11986 examples [00:02, 6291.80 examples/s]
Generating train examples...: 24000 examples [00:03, 8740.29 examples/s]
                                                                        
Shuffling C:\Users\17828\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteKCHA1E\imdb_reviews-train.tfrecord*...:   0%|          | 0/25000 [00:00<?, ? examples/s]
Shuffling C:\Users\17828\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteKCHA1E\imdb_reviews-train.tfrecord*...:  37%|███▋      | 9236/25000 [00:00<00:00, 92077.11 examples/s]
Shuffling C:\Users\17828\tensorflow_datasets\imdb_reviews\plain_text\1.0.0.incompleteKCHA1E\imdb_reviews-train.tfrecord*...:  77%|███████▋  | 19276/25000 [00:00<00:00, 96567.10 example

Dataset imdb_reviews downloaded and prepared to C:\Users\17828\tensorflow_datasets\imdb_reviews\plain_text\1.0.0. Subsequent calls will reuse this data.
tf.Tensor(
[b"This was an absolutely terrible movie. Don't be lured in by Christopher Walken or Michael Ironside. Both are great actors, but this must simply be their worst role in history. Even their great acting could not redeem this movie's ridiculous storyline. This movie is an early nineties US propaganda piece. The most pathetic scenes were those when the Columbian rebels were making their cases for revolutions. Maria Conchita Alonso appeared phony, and her pseudo-love affair with Walken was nothing but a pathetic emotional plug in a movie that was devoid of any real meaning. I am disappointed that there are movies like this, ruining actor's like Christopher Walken's good name. I could barely sit through it."
 b'I have been known to fall asleep during films, but this is usually due to a combination of things including, really tir

In [7]:
# Section: 构建模型
'''
神经网络由堆叠的层来构建，这需要从三个主要方面来进行体系结构决策

- 如何表示文本
- 模型有多少层
- 每个层有多少个隐层单元

本示例 输入数据由句子组成 预测的标签为0和1

表示文本的一种方式是将句子转换为嵌入向量。使用一个预训练文本嵌入向量作为首层，这将具有三个优点-
- 不必担心文本预处理
- 可以从迁移学习中受益
- 嵌入具有固定长度，更易于处理

使用来自Tensorflow Hub的预训练文本嵌入向量模型，名称google/nnml-en-dim50/2

让我们首先创建一个使用Tensorflow Hub模型嵌入语句的keras层，并在几个输入样本中进行尝试，
请注意无论输入文本的长度如何，嵌入层输出的形状都是：（Num_examples, embedding_dimension）


'''
embedding = "https://tfhub.dev/google/nnlm-en-dim50/2"   # 预训练文本嵌入向量模型
hub_layer = hub.KerasLayer(embedding, input_shape=[], dtype=tf.string, trainable=True)
hub_layer(train_examples_batch[:3])


<tf.Tensor: shape=(3, 50), dtype=float32, numpy=
array([[ 0.5423195 , -0.0119017 ,  0.06337538,  0.06862972, -0.16776837,
        -0.10581174,  0.16865303, -0.04998824, -0.31148055,  0.07910346,
         0.15442263,  0.01488662,  0.03930153,  0.19772711, -0.12215476,
        -0.04120981, -0.2704109 , -0.21922152,  0.26517662, -0.80739075,
         0.25833532, -0.3100421 ,  0.28683215,  0.1943387 , -0.29036492,
         0.03862849, -0.7844411 , -0.0479324 ,  0.4110299 , -0.36388892,
        -0.58034706,  0.30269456,  0.3630897 , -0.15227164, -0.44391504,
         0.19462997,  0.19528408,  0.05666234,  0.2890704 , -0.28468323,
        -0.00531206,  0.0571938 , -0.3201318 , -0.04418665, -0.08550783,
        -0.55847436, -0.23336391, -0.20782952, -0.03543064, -0.17533456],
       [ 0.56338924, -0.12339553, -0.10862679,  0.7753425 , -0.07667089,
        -0.15752277,  0.01872335, -0.08169781, -0.3521876 ,  0.4637341 ,
        -0.08492756,  0.07166859, -0.00670817,  0.12686075, -0.19326553,
 

In [8]:

# SECTION: 构建完整模型

'''
第一层是Tensorflow Hub层，此层使用预训练的SaveModel将句子映射到其嵌入向量，
您使用的预训练文本嵌入向量模型 (google/nnlm-en-dim50/2) 可将句子拆分为词例，嵌入每个词例，然后组合嵌入向量。生成的维度是：(num_examples, embedding_dimension)。对于此 NNLM 模型，embedding_dimension 是 50。

该定长输出向量通过一个有16个隐层单元的全连接层（Dense）进行管道传输

最后一层与单个输出节点紧密相连，使用Sigmoid激活函数，其函数值为介于0与1之间的浮点数，表示概率或置信水平
'''
model = tf.keras.Sequential()
model.add(hub_layer)
model.add(tf.keras.layers.Dense(16, activation='relu'))
model.add(tf.keras.layers.Dense(1))

model.summary()


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 keras_layer (KerasLayer)    (None, 50)                48190600  
                                                                 
 dense (Dense)               (None, 16)                816       
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 48191433 (183.84 MB)
Trainable params: 48191433 (183.84 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [9]:
# Section:损失函数与优化器

model.compile(optimizer='adam',
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])


In [10]:
# SECTION: 训练模型

'''
使用包含512个样本的mini-batch对模型进行10个周期的训练，也就是x_train和y_train张量中对所有样本进行10次迭代，
在训练时，检测模型在验证集的10000个样本上的损失和准确率
'''

history = model.fit(train_data.shuffle(10000).batch(512),epochs=10, validation_data=validation_data.batch(512),
                    verbose=1)

Epoch 1/10
30/30 [==============================] - 10s 322ms/step - loss: 0.6399 - accuracy: 0.5701 - val_loss: 0.5690 - val_accuracy: 0.6700
Epoch 2/10
30/30 [==============================] - 9s 316ms/step - loss: 0.4933 - accuracy: 0.7426 - val_loss: 0.4580 - val_accuracy: 0.7873
Epoch 3/10
30/30 [==============================] - 9s 318ms/step - loss: 0.3640 - accuracy: 0.8499 - val_loss: 0.3766 - val_accuracy: 0.8344
Epoch 4/10
30/30 [==============================] - 9s 318ms/step - loss: 0.2644 - accuracy: 0.8995 - val_loss: 0.3340 - val_accuracy: 0.8542
Epoch 5/10
30/30 [==============================] - 10s 319ms/step - loss: 0.1944 - accuracy: 0.9328 - val_loss: 0.3135 - val_accuracy: 0.8635
Epoch 6/10
30/30 [==============================] - 9s 318ms/step - loss: 0.1420 - accuracy: 0.9572 - val_loss: 0.3071 - val_accuracy: 0.8668
Epoch 7/10
30/30 [==============================] - 9s 316ms/step - loss: 0.1023 - accuracy: 0.9719 - val_loss: 0.3099 - val_accuracy: 0.8677
Epoc

In [11]:
# Section:评估模型
'''
将返回两个值，损失值（一个表示误差的数字，值越低越好）与准确率
'''

results = model.evaluate(test_data.batch(512), verbose=2)

for name, value in zip(model.metrics_names, results):
    print("%s: %.3f" % (name, value))

49/49 - 3s - loss: 0.3711 - accuracy: 0.8514 - 3s/epoch - 54ms/step
loss: 0.371
accuracy: 0.851
